# Section 1: Seagrass Detection

This is Section 1 of the Seagrass Detection Code, which aims to find all seagrass within a given orthomosaic using color detection. It consists of 6 sections: Preparation, Cropping, Reflection Removal, Seagrass Detection, Selection Refining, and Seagrass Area Calculation.  
Its outcome is a binary image in which all the seagrass is highlighted in white. This binary image can then be georeferenced in 'Section 2: Georeferencing' and used as raster, for example in QGIS. Further, this section calculates the area of the detected seagrass based on the pixel size, which is computed in Section 2.    
In the beginning of every subsection, the input and final output is mentioned, and a brief explanation of the code is given. 

## Part 0: Preparation

*Overview of the different libraries and their functions:*  
cv2 - short for Computer Vision, image processing library  
numpy - general data processing library  
csv - short for Comma Seperated Values, used to save the coordinates resulting from the cropping step  
os - used to access and save images to the specified directories    
shapely - for geometric shapes, used to extract the area of interest  
tqdm - used to display a progress bar for parts 2 of the code, which has a significant running time

In [1]:
# import the necessary libraries
import cv2
import numpy as np
import csv
import os
from shapely import Polygon
from tqdm import tqdm

In [5]:
# define the source path, results path, and image name
# saving the image name as a variable allows for clear storage of the preliminary results
source_path = r"C:\copy\your\source\path\here"
results_path = r"C:\copy\your\results\path\here"
image_name = "put_image_name_here"

# save the image in a variable and check that it has loaded correctly 
image = cv2.imread(os.path.join(source_path, f"{image_name}.tif"))
if image is None:
    print("Error: Image not loaded correctly. \nCheck that there are no typos and that the image name does not include '.tif'."
         f"\nCurrently, your path is: {os.path.join(source_path, f"{image_name}.tif")}")
else:
    print("The image was loaded correctly.")

Error: Image not loaded correctly. 
Check that there are no typos and that the image name does not include '.tif'.
Currently, your path is: C:\copy\your\source\path\here\put_image_name_here.tif


Throughout the code, two images are shown within the same window several times.  
If you want the images to show up on top of each other: axis = 0, and if you want them to be next to each other: axis = 1.

In [24]:
axis = 1

## Part 1: Cropping the tif to the area of interest

*Input: original tif (image)  
Output: cropped image (cropped_image)*  
This part of the code allows you to select an area of interest (the sea), based on which a mask is created. The mask is then overlayed with the original image and only the area within the mask is used for futher processing. 

In [17]:
# select points around the area of interest based on which the mask will be created (crops out the land area and keeps the sea)
def main():
 
    # open a csv file to save the coordinates
    with open("coordinates_shoreline.csv", "w") as f:
            
        # display instructions in the console
        print("Select a series of points by clicking on the image.")
        print("Press 'Enter' to finish the selection.")
        
        # create a window for displaying the image
        cv2.namedWindow("Point Selection")
        cv2.imshow("Point Selection", image)
        
        # list to store the coordinate points
        green_points = []
        
        # wait for the user to select points by clicking
        def on_mouse_click(event, x, y, flags, param):
            if event == cv2.EVENT_LBUTTONDOWN:
     
                # add the point to the list of green points
                green_points.append((x, y))
                
                # display the clicked point in green
                cv2.circle(image, (x, y), 2, (0, 255, 0), -1)
                
                # print the coordinates
                print("Coordinates:", (x, y))
        
                # save the coordinates in the csv file
                with open("coordinates_shoreline.csv", mode="w", newline="") as file:
                    writer = csv.writer(file)
                    writer.writerows(green_points)
                
                # draw a red line between the last two green points
                if len(green_points) >= 2:
                    pt1 = green_points[-2]
                    pt2 = green_points[-1]
                    cv2.line(image, (pt1[0], pt1[1]), (pt2[0], pt2[1]), (0, 0, 255), 2)
                
                # Update the image display
                cv2.imshow("Point Selection", image)

        cv2.setMouseCallback("Point Selection", on_mouse_click)
        
        # Wait for the user to press the 'Enter' key to finish the selection
        while True:
            key = cv2.waitKey(0)
            if key == 13:  # 'Enter' key
                break
        
        cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

Select a series of points by clicking on the image.
Press 'Enter' to finish the selection.
Coordinates: (393, 38)
Coordinates: (1302, 843)
Coordinates: (2590, 1691)
Coordinates: (3750, 2346)
Coordinates: (5099, 2731)
Coordinates: (6174, 2847)
Coordinates: (6201, 1040)
Coordinates: (6128, 970)
Coordinates: (3788, 215)
Coordinates: (1676, 0)
Coordinates: (439, 42)


In [18]:
# create a mask based on the selected coordinates and crop the image

# open the coordinates as a list and transform into a numpy array 
with open("coordinates_shoreline.csv", "r") as file:
    coord = list(csv.reader(file, quoting=csv.QUOTE_NONNUMERIC))
pts = np.reshape((np.array(coord, np.int32)), (-1, 1, 2))

# create a canvas (a black background) in the same shape as the original image, the mask will be saved on this canvas  
canvas = np.zeros((np.shape(image)), dtype = 'uint8')

# create a mask based on the previously selected points, polylines connects the points and fillConvexPoly then creates a polygon
# (255, 255, 255) means the mask will be displayed in white
mask = cv2.polylines(canvas, pts, True, (255,255,255))
cv2.fillConvexPoly(mask, pts, (255, 255, 255))

# overlay the mask and the original image using bitwise_and, only the area within the mask will be used in the following parts
cropped_image = cv2.bitwise_and(mask, image)

# show the mask next to the cropped area 
# if they show up next to each other when you want them stacked or vice versa, refer back to 'Part 0: Preparation' and change the axis value
mask_and_cropped = np.concatenate((mask, cropped_image), axis = axis)
cv2.imshow('Mask and Cropped Image', mask_and_cropped)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Part 2: Removing the Reflection

*input: cropped image (cropped_image)  
output: image without reflection (image_without_reflection)*  
This section of the code detects the reflection in the area of interest and then removes it. First, the image is converted from BGR to HSV (blue green red to hue saturation value), of which we are interested in the saturation specifically, as it is very high in when there is reflection. A modifiable threshold is given: all the pixels with a value within that threshold (meaning all pixels lighter than the minimum value) are then marked as reflections. The area around the relfections is then used to get the mean BGR colors, which are in turn applied to the reflection. A tif file in which the selected pixels are highlighted is created to allow for user interaction and easier adatpation of the code, and a tif file without the reflection is the final result of this section. 

In [19]:
# convert the BGR image to HSV and split to access the saturation only
imghsv = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2HSV)
(h, s, v) = cv2.split(imghsv)

# create a binary image based on the saturation: the first value can be modified, the second (255) should remain the same
# Increasing the first number will increase the number of reflections detected, decreasing it will decrease the number of reflections detected
_, binary = cv2.threshold(s, 100, 255, cv2.THRESH_BINARY_INV)
cropped_binary = cv2.bitwise_and(binary, mask[:,:,0])
reflection_vs_original = np.concatenate((cropped_image[:,:,0], cropped_binary), axis=axis)
cv2.imshow('Detected Reflection', reflection_vs_original)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [20]:
#define function that replaces white pixels with average colour around them
            
def replace_white_color(image, contours): 
    i = 0
    for contour in contours: 
        #Create a blank mask for each contour
        white_mask = np.zeros(cropped_image.shape[:2], dtype=np.uint8)
        
        # Draw contour onto the mask
        cv2.drawContours(white_mask, [contour], -1, 255, thickness=cv2.FILLED)

        # Dilate the mask to include a border around the contour
        kernel = np.ones((5, 5), np.uint8)
        dilated_white_mask = cv2.dilate(white_mask, kernel, iterations=1)

        # Calculate the area covered by the dilated mask excluding the original mask
        mask_diff = cv2.subtract(dilated_white_mask, white_mask)

        # Apply the mask difference to the original image and calculate the mean color
        mean_color = cv2.mean(image, mask=mask_diff)[:3]            
        mean_color = tuple([int(c) for c in mean_color])
    
        # Replace the color in the modified image only where white_mask is 255
        image[dilated_white_mask == 255] = mean_color
        i += 1
        pbar.update(1)
    print("Number of detected and processed contours:", len(contours))
    
    return image

In [21]:
# Find all the contours (white zones) in the binary image
contours, _ = cv2.findContours(cropped_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Draw the contours on the original image and run the reflection removal code
image_with_contours = cropped_image.copy()
cv2.drawContours(image_with_contours, contours, -1, (0, 255, 255), -1)
print(len(contours), 'contours were detected. Based on previous experience removing the reflection will take approximately', round(len(contours)/1000*6.7, 0), 'minutes.')

2013 contours were detected. Based on previous experience removing the reflection will take approximately 13.0 minutes.


In [22]:
# Remove the reflection by replacing white contours with the average color of the area around them
# displays a progress bar
# if you run this code multiple times, there might be an issue with the progress bar (for every perecentage a new line appears) 
# if this happens, run: pbar.close() before the rest of the code
#pbar.close()
pbar = tqdm(total = len(contours), desc = 'Processed Contours', colour = 'cyan')
image_filled = replace_white_color(cropped_image.copy(), contours)
pbar.close()

Processed Contours: 100%|██████████████████████████████████████████████████████████| 2013/2013 [02:48<00:00, 11.97it/s]

Number of detected and processed contours: 2013


In [26]:
# Display the image with the contours of white pixels highlighted in yellow
concatenated_image = np.concatenate((image_with_contours, image_filled), axis=axis) 

cv2.imshow("Detected Reflection (yellow) and Image without Reflection", concatenated_image)
cv2.imwrite(os.path.join(results_path, f"{image_name}_withoutReflection.tif"), image_filled)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Part 3: Identifying the Seagrass

*input: cropped image without reflection (image_without_reflection)  
output: binary image where seagrass appears in white (dbo_binary)*  
Part 3 of the code aims to select all the seagrass within the cropped image without reflection based on color detection. First, the image is converted from RGB (red green blue) to HSV (hue saturation value), for which the threshold values can then be set. A window opens up in which all the detected areas are outlined. As the values differ significantly from image to image, another window opens up in the next step, from which you can select points for which you would like to know the HSV values. You can then modify the thresholds and repeat this several times until you are happy with the selection.  
At the end of this subsection, the outlines will most likely still contain some noise and some holes within your patches, which will be fixed in 'Part 4: Refining Selection'.

In [27]:
# load the image without reflection
image_without_reflection = cv2.imread(os.path.join(results_path, f"{image_name}_withoutReflection.tif"), image_filled)
if image_without_reflection is None:
    print("The image did not load correctly.")

# define the function that will select all dark blue objects within the given thresholds
def segment_dark_blue_objects(image, lower_bound, upper_bound):
    hsv_image = cv2.cvtColor(image_without_reflection, cv2.COLOR_BGR2HSV)
    dark_blue_mask = cv2.inRange(hsv_image, lower_bound, upper_bound)
    dark_blue_objects = cv2.bitwise_and(image, image, mask=dark_blue_mask)
    return dark_blue_objects

# define the function that will draw outlines around the detected dark blue objects
def draw_outlines(binary_image):
    # find contours of binary image
    contours, _ = cv2.findContours(binary_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # draw contours
    image_with_contours = image_without_reflection.copy()
    cv2.drawContours(image_with_contours, contours, -1, (0, 255, 255), 2)

    # display outlines and save the image to your device
    cv2.imshow('Outlines', image_with_contours)
    cv2.imwrite(os.path.join(results_path, f'{image_name}_outlines.tif'), image_with_contours)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [80]:
# Segment dark blue objects
blue_min = np.array([97, 0, 65])
blue_max = np.array([110, 255, 200])
dark_blue_objects = segment_dark_blue_objects(image_without_reflection, blue_min, blue_max)

# create binary of the dark blue objects (dbo) for further processing
dbo_grayscale = cv2.cvtColor(dark_blue_objects, cv2.COLOR_RGB2GRAY)
_, dbo_binary = cv2.threshold(dbo_grayscale, 1, 255, cv2.THRESH_BINARY)

# draw outlines around the selected objects
draw_outlines(dbo_binary)

In [79]:
# open the outlined image
image_with_contours = cv2.imread(os.path.join(results_path, f'{image_name}_outlines.tif'))
if image_with_contours is None:
    print("The image did not load correctly.")

# create an empty list to store the selected points
seagrass_points = []

# define the function which fills the list with the coordinates of points you've selected
def on_mouse_click(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        # Add the point to the seagrass list
        seagrass_points.append((y, x))
                
        # Display the clicked point in red
        cv2.circle(image, (y, x), 2, (0, 0, 255), -1)

# create a window in which the outlines image will be displayed, from which you can select the points for which you want the HSV values
cv2.namedWindow("Seagrass Point Selection")
cv2.imshow("Seagrass Point Selection", image_with_contours)
cv2.setMouseCallback("Seagrass Point Selection", on_mouse_click)
cv2.waitKey(0)
cv2.destroyAllWindows()

# print the HSV values of the selected points
HSV = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2HSV)
for point in seagrass_points:
    print(HSV[point])

[102 236  67]
[106 228  67]
[104 223  72]
[103 212  71]
[104 223  72]
[105 221  67]
[106 214  69]
[107 212  66]
[105 226  70]
[104 192  69]
[106 161  87]


## Part 4: Refining the Selection

In [62]:
# define the function which removes noise
def remove_noise(image, kernel_size, iterations):
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(kernel_size, kernel_size))
    image_without_noise = cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel, iterations = iterations)
    return image_without_noise
    
# define the function which fill holes in the seagrass patches
def fill_holes(image, kernel_size, iterations):
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(kernel_size, kernel_size))
    image_with_filled_patches = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel, iterations=iterations)
    return image_with_filled_patches

In [82]:
# remove noise and fill up the holes in the seagrass patches
# the kernel size and amount of iterations can be changed

image_without_noise = remove_noise(dbo_binary, 3, 8)
image_with_filled_patches = fill_holes(image_without_noise, 3, 10)

draw_outlines(image_with_filled_patches)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [83]:
# when happy with the result, save the refined (binary) image to the results folder
cv2.imwrite(os.path.join(results_path, f"{image_name}_SeagrassBinary.tif"), image_with_filled_patches)

True

Congratulations, this part of the code is now done! To continue working with the seagrass areas, please refer to the protocol and move on to Section 2, in which the binary tif is georeferenced to contain the same geographical information as the input orthomosaic. It can then be used as a raster, for example in QGIS. Return here once you're done with section 2 to calculate the area.

## Part 6: Quantifying the Seagrass Areas

*Input: Binary picture of the detected seagrass, pixel size  
Output: Calculated Seagrass Area*  
This section of the code calculates the detected seagrass area based on the pixel size, which is given through Section 2: Georeferencing. At a flying height of 100 metres, every pixel is approxiamtely equivalent to 5x5 cm. Based on the amount of detected pixels, the area is then calculated. 

In [3]:
# define the function that computes the total amount of white pixels and then calculates the area in m2 and ha based on that

def calculate_white_area(image, pixel_size):
 
    # Check if the image was loaded correctly
    if image is None:
        print("Error: Image not loaded correctly.")
        return

    # Count the number of white pixels (value 255)
    white_pixel_count = cv2.countNonZero(image[:,:,0])

    # Calculate the pixel area based on the pixel size
    area_per_pixel_in_cm2 = pixel_size
    area_per_pixel_in_m2 = area_per_pixel_in_cm2 / 10000

    # Total area in square metres
    area_m2 = white_pixel_count * area_per_pixel_in_m2

    # Convert area to hectares
    area_hectares = area_m2 / 10000

    # Print the results
    print(f"Number of white pixels: {white_pixel_count}")
    print(f"Area covered by one pixel: {round(area_per_pixel_in_cm2, 2)} square centimetres")
    print(f"Covered area: {round(area_m2, 2)} square meters")
    print(f"Covered area: {round(area_hectares, 4)} hectares")

In [4]:
# Define inputs and run function
seagrass_image = cv2.imread(os.path.join(results_path, f"{image_name}_SeagrassBinary.tif"))

# pixel size in centimetres, copy from Part 5: Georeferencing
pixel_size = 25.254630790658826

# Calculate and display the white area
calculate_white_area(seagrass_image, pixel_size)

Number of white pixels: 3577661
Area covered by one pixel: 25.25 square centimetres
Covered area: 9035.25 square meters
Covered area: 0.9035 hectares
